### Trabalho de Conclusão de Curso
### Pós Graduação Ciência de Dados e Inteligência Artificial - PUCRS

### UTILIZANDO TÉCNICAS DE MACHINE LEARNING NA PRECIFICAÇÃO DE TARIFAS DE ARRECADAÇÃO TRIBUTÁRIA NO BRASIL

### Michel Dourado

### Parte 4 - Machine Learning

In [2]:
# Import 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.linear_model import LinearRegression, Ridge, LassoCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics 
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x : '%.2f' % x)
%matplotlib inline

In [3]:
#Vamos continuar trabalhando com o dataframe do pandas gerando após os tratamentos realizados na fase de preparação dos dados
dados = pd.read_csv('df_pd_pre_processamento_vf.csv')

In [4]:
#dados[['Quantidade_media_guias','Volume_medio_arrecadado','Tarifa_media', 'Esfera_ESTADUAL', 'Esfera_MUNICIPAL']]
dados

,Quantidade_media_guias,Volume_medio_arrecadado,Tarifa_media,UF_AC,UF_AL,UF_AM,UF_AP,UF_BA,UF_CE,UF_ES,...,UF_RN,UF_RO,UF_RR,UF_RS,UF_SC,UF_SE,UF_SP,UF_TO,Esfera_ESTADUAL,Esfera_MUNICIPAL
0,9.87,11.81,0.93,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,7.56,9.21,0.93,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,8.41,8.65,0.93,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,5.49,0.00,0.93,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4.32,0.00,0.93,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2938,9.65,11.11,0.02,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2939,8.23,4.02,0.02,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2940,10.01,11.93,0.01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2941,7.49,4.89,0.01,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
dados.columns

Index(['Quantidade_media_guias', 'Volume_medio_arrecadado', 'Tarifa_media',
       'UF_AC', 'UF_AL', 'UF_AM', 'UF_AP', 'UF_BA', 'UF_CE', 'UF_ES', 'UF_GO',
       'UF_MA', 'UF_MG', 'UF_MS', 'UF_MT', 'UF_PA', 'UF_PB', 'UF_PE', 'UF_PI',
       'UF_PR', 'UF_RJ', 'UF_RN', 'UF_RO', 'UF_RR', 'UF_RS', 'UF_SC', 'UF_SE',
       'UF_SP', 'UF_TO', 'Esfera_ESTADUAL', 'Esfera_MUNICIPAL'],
      dtype='object')

In [6]:
#filtrando as colunas relevantes
features1 = ['Quantidade_media_guias','Volume_medio_arrecadado', 'Esfera_ESTADUAL', 'Esfera_MUNICIPAL', 'UF_AC', 'UF_AL', 'UF_AM', 'UF_AP', 'UF_BA', 'UF_CE', 'UF_ES', 'UF_GO',
       'UF_MA', 'UF_MG', 'UF_MS', 'UF_MT', 'UF_PA', 'UF_PB', 'UF_PE', 'UF_PI',
       'UF_PR', 'UF_RJ', 'UF_RN', 'UF_RO', 'UF_RR', 'UF_RS', 'UF_SC', 'UF_SE',
       'UF_SP', 'UF_TO']
features2 = ['Quantidade_media_guias','Volume_medio_arrecadado', 'Esfera_ESTADUAL', 'Esfera_MUNICIPAL']

features3 = ['Quantidade_media_guias','Volume_medio_arrecadado']

In [7]:
#preparando a variável target
col_target ='Tarifa_media'

In [8]:
#gerando dados de treino
df_treino = dados.sample(frac = 0.7, random_state = 101)

In [9]:
#selecionando tudo que não está no dataset de treino e atribuindo ao dataset de teste
df_teste = dados.loc[~dados.index.isin(df_treino.index)]

In [10]:
#shape dos datasets
print(df_treino.shape)
print(df_teste.shape)

(2060, 31)
(883, 31)


In [11]:
X_treino1 = df_treino[features1]
X_treino2 = df_treino[features2]
X_treino3 = df_treino[features3]

In [12]:
y_treino = df_treino[col_target]

In [13]:
X_teste1 = df_teste[features1]
X_teste2 = df_teste[features2]
X_teste3 = df_teste[features3]

In [14]:
y_teste = df_teste[col_target]

## Modelo de Regressão Linear Múltipla

In [15]:
#Criando um regressor
reg_v1_1 = LinearRegression()
reg_v1_2 = LinearRegression()
reg_v1_3 = LinearRegression()

In [16]:
modelo_v1_1 = reg_v1_1.fit(X_treino1,y_treino)
modelo_v1_2 = reg_v1_2.fit(X_treino2,y_treino)
modelo_v1_3 = reg_v1_3.fit(X_treino3,y_treino)

In [17]:
y_pred1 = modelo_v1_1.predict(X_teste1)
y_pred2 = modelo_v1_2.predict(X_teste2)
y_pred3 = modelo_v1_3.predict(X_teste3)

In [19]:
# Calcula a performance do modelo
df_prev1 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':y_pred1})
df_prev1["Tarifa Média Real"] = np.expm1(df_prev1["Tarifa Média Real"])
df_prev1["Tarifa Média Prevista"] = np.expm1(df_prev1["Tarifa Média Prevista"])
df_prev1

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.57
3,1.54,0.57
10,1.53,0.69
11,1.53,0.65
13,1.52,0.59
...,...,...
2932,0.03,0.47
2936,0.02,0.86
2937,0.02,0.37
2941,0.01,0.72


In [20]:
df_prev2 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':y_pred2})
df_prev2["Tarifa Média Real"] = np.expm1(df_prev2["Tarifa Média Real"])
df_prev2["Tarifa Média Prevista"] = np.expm1(df_prev2["Tarifa Média Prevista"])
df_prev2

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.65
3,1.54,0.65
10,1.53,0.63
11,1.53,0.76
13,1.52,0.67
...,...,...
2932,0.03,0.55
2936,0.02,0.84
2937,0.02,0.37
2941,0.01,0.59


In [21]:
df_prev3 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':y_pred3})
df_prev3["Tarifa Média Real"] = np.expm1(df_prev3["Tarifa Média Real"])
df_prev3["Tarifa Média Prevista"] = np.expm1(df_prev3["Tarifa Média Prevista"])
df_prev3

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.65
3,1.54,0.65
10,1.53,0.63
11,1.53,0.76
13,1.52,0.67
...,...,...
2932,0.03,0.55
2936,0.02,0.84
2937,0.02,0.37
2941,0.01,0.59


In [22]:
df_prev1.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.64
std,0.38,0.15
min,0.01,0.31
25%,0.35,0.54
50%,0.63,0.63
75%,0.97,0.74
max,1.54,1.17


In [23]:
df_prev2.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.65
std,0.38,0.12
min,0.01,0.33
25%,0.35,0.57
50%,0.63,0.65
75%,0.97,0.72
max,1.54,1.16


In [24]:
df_prev3.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.65
std,0.38,0.12
min,0.01,0.33
25%,0.35,0.57
50%,0.63,0.65
75%,0.97,0.72
max,1.54,1.16


In [25]:
# Resultados
score1_1 = modelo_v1_1.score(X_teste1, y_teste)
MAE1_1 = metrics.mean_absolute_error(y_teste, y_pred1)
MSE1_1 = metrics.mean_squared_error(y_teste, y_pred1)
RMSE1_1 = np.sqrt(metrics.mean_squared_error(y_teste, y_pred1))

score1_2 = modelo_v1_2.score(X_teste2, y_teste)
MAE1_2 = metrics.mean_absolute_error(y_teste, y_pred2)
MSE1_2 = metrics.mean_squared_error(y_teste, y_pred2)
RMSE1_2 = np.sqrt(metrics.mean_squared_error(y_teste, y_pred2))

score1_3 = modelo_v1_3.score(X_teste3, y_teste)
MAE1_3 = metrics.mean_absolute_error(y_teste, y_pred3)
MSE1_3 = metrics.mean_squared_error(y_teste, y_pred3)
RMSE1_3 = np.sqrt(metrics.mean_squared_error(y_teste, y_pred3))

In [28]:
print("### Modelo V1_1 Linear Regression ###")
print("Score:",(score1_1))
print('Mean Absolute Error:', MAE1_1)
print('Mean Squared Error:', MSE1_1)
print('Root Mean Squared Error:', RMSE1_1)
print("")
print("### Modelo V1_2 Linear Regression ###")
print("Score:",(score1_2))
print('Mean Absolute Error:', MAE1_2)
print('Mean Squared Error:', MSE1_2)
print('Root Mean Squared Error:', RMSE1_2)
print("")
print("### Modelo V1_3 Linear Regression ###")
print("Score:",(score1_3))
print('Mean Absolute Error:', MAE1_3)
print('Mean Squared Error:', MSE1_3)
print('Root Mean Squared Error:', RMSE1_3)

### Modelo V1_1 Linear Regression ###
Score: 0.10033553177152799
Mean Absolute Error: 0.18290613510439707
Mean Squared Error: 0.04829439546265297
Root Mean Squared Error: 0.2197598586244835

### Modelo V1_2 Linear Regression ###
Score: 0.0796841460255332
Mean Absolute Error: 0.18684051032849222
Mean Squared Error: 0.04940297118759266
Root Mean Squared Error: 0.22226779161091392

### Modelo V1_3 Linear Regression ###
Score: 0.08100813647914762
Mean Absolute Error: 0.1867858553461154
Mean Squared Error: 0.04933189878136377
Root Mean Squared Error: 0.2221078539389451


In [27]:
# O modelo V1_1 com todas as variáveis categóricas apresentou os melhores resultados em todas as métricas, 
#sendo o maior score e os menores erros

# Modelo de Regressão Random Forest

In [29]:
# Criando um regressor Random Forest
reg_v2_1 = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 10, random_state = 101)
reg_v2_2 = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 10, random_state = 101)
reg_v2_3 = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 10, random_state = 101)

In [30]:
# Criando o modelo
modelo_v2_1 = reg_v2_1.fit(X_treino1, y_treino)
modelo_v2_2 = reg_v2_2.fit(X_treino2, y_treino)
modelo_v2_3 = reg_v2_3.fit(X_treino3, y_treino)

In [31]:
# Fazendo previsões
previsoes_v2_1 = modelo_v2_1.predict(X_teste1)
previsoes_v2_2 = modelo_v2_2.predict(X_teste2)
previsoes_v2_3 = modelo_v2_3.predict(X_teste3)

In [32]:
# Calcula a performance do modelo
df_v2_1 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':previsoes_v2_1})
df_v2_1["Tarifa Média Real"] = np.expm1(df_v2_1["Tarifa Média Real"])
df_v2_1["Tarifa Média Prevista"] = np.expm1(df_v2_1["Tarifa Média Prevista"])
df_v2_1

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.62
3,1.54,0.58
10,1.53,0.66
11,1.53,0.80
13,1.52,0.49
...,...,...
2932,0.03,0.40
2936,0.02,0.84
2937,0.02,0.53
2941,0.01,0.73


In [33]:
# Calcula a performance do modelo
df_v2_2 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':previsoes_v2_2})
df_v2_2["Tarifa Média Real"] = np.expm1(df_v2_2["Tarifa Média Real"])
df_v2_2["Tarifa Média Prevista"] = np.expm1(df_v2_2["Tarifa Média Prevista"])
df_v2_2

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.94
3,1.54,0.66
10,1.53,0.52
11,1.53,0.80
13,1.52,0.75
...,...,...
2932,0.03,0.46
2936,0.02,0.77
2937,0.02,0.45
2941,0.01,0.61


In [34]:
# Calcula a performance do modelo
df_v2_3 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':previsoes_v2_3})
df_v2_3["Tarifa Média Real"] = np.expm1(df_v2_3["Tarifa Média Real"])
df_v2_3["Tarifa Média Prevista"] = np.expm1(df_v2_3["Tarifa Média Prevista"])
df_v2_3

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.94
3,1.54,0.66
10,1.53,0.52
11,1.53,0.80
13,1.52,0.75
...,...,...
2932,0.03,0.46
2936,0.02,0.77
2937,0.02,0.45
2941,0.01,0.61


In [35]:
df_v2_1.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.65
std,0.38,0.16
min,0.01,0.27
25%,0.35,0.55
50%,0.63,0.66
75%,0.97,0.76
max,1.54,1.03


In [36]:
df_v2_2.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.66
std,0.38,0.15
min,0.01,0.28
25%,0.35,0.56
50%,0.63,0.67
75%,0.97,0.77
max,1.54,1.08


In [37]:
df_v2_3.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.66
std,0.38,0.15
min,0.01,0.28
25%,0.35,0.56
50%,0.63,0.67
75%,0.97,0.77
max,1.54,1.08


In [38]:
# Score
score2_1 = modelo_v2_1.score(X_teste1, y_teste)
MAE2_1 = metrics.mean_absolute_error(y_teste, previsoes_v2_1)
MSE2_1 = metrics.mean_squared_error(y_teste, previsoes_v2_1)
RMSE2_1 = np.sqrt(metrics.mean_squared_error(y_teste, previsoes_v2_1))

score2_2 = modelo_v2_2.score(X_teste2, y_teste)
MAE2_2 = metrics.mean_absolute_error(y_teste, previsoes_v2_2)
MSE2_2 = metrics.mean_squared_error(y_teste, previsoes_v2_2)
RMSE2_2 = np.sqrt(metrics.mean_squared_error(y_teste, previsoes_v2_2))

score2_3 = modelo_v2_3.score(X_teste3, y_teste)
MAE2_3 = metrics.mean_absolute_error(y_teste, previsoes_v2_3)
MSE2_3 = metrics.mean_squared_error(y_teste, previsoes_v2_3)
RMSE2_3 = np.sqrt(metrics.mean_squared_error(y_teste, previsoes_v2_3))

In [40]:
print("### Modelo V2_1 Random Forests")
print("Score:",(score2_1))
print('Mean Absolute Error:', MAE2_1)
print('Mean Squared Error:', MSE2_1)
print('Root Mean Squared Error:', RMSE2_1)
print("")
print("### Modelo V2_2 Random Forests")
print("Score:",(score2_2))
print('Mean Absolute Error:', MAE2_2)
print('Mean Squared Error:', MSE2_2)
print('Root Mean Squared Error:', RMSE2_2)
print("")
print("### Modelo V2_3 Random Forests")
print("Score:",(score2_3))
print('Mean Absolute Error:', MAE2_3)
print('Mean Squared Error:', MSE2_3)
print('Root Mean Squared Error:', RMSE2_3)

### Modelo V2_1 Random Forests
Score: 0.08725395577806883
Mean Absolute Error: 0.1837030153127719
Mean Squared Error: 0.048996620377177905
Root Mean Squared Error: 0.22135180229033127

### Modelo V2_2 Random Forests
Score: 0.04714046398716376
Mean Absolute Error: 0.1890905321495705
Mean Squared Error: 0.05114993075493741
Root Mean Squared Error: 0.22616350447173703

### Modelo V2_3 Random Forests
Score: 0.04714046398716387
Mean Absolute Error: 0.1890905321495705
Mean Squared Error: 0.0511499307549374
Root Mean Squared Error: 0.226163504471737


In [41]:
# O modelo V2_1 com todas as variáveis categóricas novamente apresentou os melhores resultados em todas as métricas, 
#sendo o maior score e os menores erros

# Modelo de Regressão SVR

In [42]:
svr1 = svm.SVR(gamma='auto')
svr2 = svm.SVR(gamma='auto')
svr3 = svm.SVR(gamma='auto')

In [43]:
svr1.fit(X_treino1, y_treino)
svr2.fit(X_treino2, y_treino)
svr3.fit(X_treino3, y_treino)

SVR(gamma='auto')

In [44]:
previsoes_v3_1 = svr1.predict(X_teste1)
previsoes_v3_2 = svr2.predict(X_teste2)
previsoes_v3_3 = svr3.predict(X_teste3)

In [45]:
# Calcula a performance do modelo
df_v3_1 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':previsoes_v3_1})
df_v3_1["Tarifa Média Real"] = np.expm1(df_v3_1["Tarifa Média Real"])
df_v3_1["Tarifa Média Prevista"] = np.expm1(df_v3_1["Tarifa Média Prevista"])
df_v3_1

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.57
3,1.54,0.69
10,1.53,0.72
11,1.53,0.72
13,1.52,0.55
...,...,...
2932,0.03,0.39
2936,0.02,0.91
2937,0.02,0.40
2941,0.01,0.68


In [46]:
# Calcula a performance do modelo
df_v3_2 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':previsoes_v3_2})
df_v3_2["Tarifa Média Real"] = np.expm1(df_v3_2["Tarifa Média Real"])
df_v3_2["Tarifa Média Prevista"] = np.expm1(df_v3_2["Tarifa Média Prevista"])
df_v3_2

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.78
3,1.54,0.74
10,1.53,0.69
11,1.53,0.82
13,1.52,0.73
...,...,...
2932,0.03,0.42
2936,0.02,0.77
2937,0.02,0.39
2941,0.01,0.38


In [47]:
# Calcula a performance do modelo
df_v3_3 = pd.DataFrame({'Tarifa Média Real':y_teste, 'Tarifa Média Prevista':previsoes_v3_3})
df_v3_3["Tarifa Média Real"] = np.expm1(df_v3_3["Tarifa Média Real"])
df_v3_3["Tarifa Média Prevista"] = np.expm1(df_v3_3["Tarifa Média Prevista"])
df_v3_3

,Tarifa Média Real,Tarifa Média Prevista
1,1.54,0.81
3,1.54,0.74
10,1.53,0.70
11,1.53,0.79
13,1.52,0.70
...,...,...
2932,0.03,0.41
2936,0.02,0.84
2937,0.02,0.38
2941,0.01,0.43


In [48]:
df_v3_1.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.64
std,0.38,0.18
min,0.01,0.16
25%,0.35,0.52
50%,0.63,0.65
75%,0.97,0.78
max,1.54,1.26


In [49]:
df_v3_2.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.66
std,0.38,0.17
min,0.01,0.21
25%,0.35,0.54
50%,0.63,0.69
75%,0.97,0.77
max,1.54,1.41


In [50]:
df_v3_3.describe()

,Tarifa Média Real,Tarifa Média Prevista
count,883.00,883.00
mean,0.67,0.66
std,0.38,0.17
min,0.01,0.21
25%,0.35,0.55
50%,0.63,0.69
75%,0.97,0.79
max,1.54,1.31


In [51]:
# Score
score3_1 = svr1.score(X_teste1, y_teste)
MAE3_1 = metrics.mean_absolute_error(y_teste, previsoes_v3_1)
MSE3_1 = metrics.mean_squared_error(y_teste, previsoes_v3_1)
RMSE3_1 = np.sqrt(metrics.mean_squared_error(y_teste, previsoes_v3_1))

score3_2 = svr2.score(X_teste2, y_teste)
MAE3_2 = metrics.mean_absolute_error(y_teste, previsoes_v3_2)
MSE3_2 = metrics.mean_squared_error(y_teste, previsoes_v3_2)
RMSE3_2 = np.sqrt(metrics.mean_squared_error(y_teste, previsoes_v3_2))

score3_3 = svr3.score(X_teste3, y_teste)
MAE3_3 = metrics.mean_absolute_error(y_teste, previsoes_v3_3)
MSE3_3 = metrics.mean_squared_error(y_teste, previsoes_v3_3)
RMSE3_3 = np.sqrt(metrics.mean_squared_error(y_teste, previsoes_v3_3))

In [52]:
print("### Modelo V3_1 SVR")
print("Score:",(score3_1))
print('Mean Absolute Error:', MAE3_1)
print('Mean Squared Error:', MSE3_1)
print('Root Mean Squared Error:', RMSE3_1)
print("")
print("### Modelo V3_2 SVR")
print("Score:",(score3_2))
print('Mean Absolute Error:', MAE3_2)
print('Mean Squared Error:', MSE3_2)
print('Root Mean Squared Error:', RMSE3_2)
print("")
print("### Modelo V3_3 SVR")
print("Score:",(score3_3))
print('Mean Absolute Error:', MAE3_3)
print('Mean Squared Error:', MSE3_3)
print('Root Mean Squared Error:', RMSE3_3)

### Modelo V3_1 SVR
Score: 0.10528941826655747
Mean Absolute Error: 0.18088053494010542
Mean Squared Error: 0.04802846859556312
Root Mean Squared Error: 0.2191539837547178

### Modelo V3_2 SVR
Score: 0.05961882487407266
Mean Absolute Error: 0.1865997844752258
Mean Squared Error: 0.050480086700092464
Root Mean Squared Error: 0.2246777396630393

### Modelo V3_3 SVR
Score: 0.04877008281432493
Mean Absolute Error: 0.1874000009098794
Mean Squared Error: 0.051062452079418216
Root Mean Squared Error: 0.2259700247365084


In [53]:
# O modelo V3_1 com todas as variáveis categóricas novamente apresentou os melhores resultados em todas as métricas, 
#sendo o maior score e os menores erros

### Quadro resumo comparativo dos resultados

In [54]:
pd.set_option('display.float_format', lambda x : '%.4f' % x)

modelos = ['Regressão Linear', 'Random Forest', 'Support Vector Regressor']
MAE = [MAE1_1,MAE2_1,MAE3_1]
MSE = [MSE1_1,MSE2_1,MSE3_1]
RMSE = [RMSE1_1, RMSE2_1, RMSE3_1]
SCORE = [score1_1, score2_1, score3_1]
results = pd.DataFrame(index = modelos)
results['MAE'] = MAE
results['MSE'] = MSE
results['RMSE'] = RMSE
results['SCORE'] = SCORE
results

,MAE,MSE,RMSE,SCORE
Regressão Linear,0.1829,0.0483,0.2198,0.1003
Random Forest,0.1837,0.0490,0.2214,0.0873
Support Vector Regressor,0.1809,0.0480,0.2192,0.1053


In [55]:
# o modelo escolhido foi o representado pelo algoritmo Support Vector Regressor - SVR, pois levou ligeira vantagem em todas 
# as métricas, com o menor erro absoluto - MAE, o menor erro quadrático - MSE, o menor erro RMSE e o maior SCORE. 

# Fim